# Modeling COVID-19 with bayesian hierarchical models

In [134]:
# Imports
import json
from pathlib import Path

import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading data

* ECDC (via Our World in Data)
* Our World in Data Testing coverage
* UN regions
* UN population estimates (September 2019)

In [162]:
covid_cases = 'data/covid-19-cases.csv'
covid_tests = 'data/covid-19-tests-country.csv'
un_regions = 'data/un_regions.csv'
un_pop = 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv'

df_cases = pd.read_csv(covid_cases)
df_tests = pd.read_csv(covid_tests)
df_regions = pd.read_csv(un_regions)
df_pop = pd.read_csv(un_pop)

In [136]:
df_cases.shape

(5758, 6)

In [137]:
df_cases.head(10)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0
3,2020-01-03,Afghanistan,0,0,0,0
4,2020-01-04,Afghanistan,0,0,0,0
5,2020-01-05,Afghanistan,0,0,0,0
6,2020-01-06,Afghanistan,0,0,0,0
7,2020-01-07,Afghanistan,0,0,0,0
8,2020-01-08,Afghanistan,0,0,0,0
9,2020-01-09,Afghanistan,0,0,0,0


In [138]:
df_tests.shape

(64, 4)

In [139]:
df_tests.head(10)

,Entity,Code,Year,Total COVID-19 tests
0,Armenia,ARM,55,694
1,Australia - Government of the Australian Capit...,NaN,56,1391
2,Australia - New South Wales,NaN,56,30244
3,Austria,AUT,56,10278
4,Bahrain,BHR,56,13553
5,Belarus,BLR,55,16000
6,Belgium,BEL,51,4225
7,Brazil,BRA,55,2927
8,Canada,CAN,56,38482
9,Canada - Alberta,NaN,55,10598


In [140]:
df_regions.shape

(249, 15)

In [141]:
df_regions.head(10)

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS),Developed / Developing Countries
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZA,NaN,NaN,NaN,Developing
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EGY,NaN,NaN,NaN,Developing
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LBY,NaN,NaN,NaN,Developing
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MAR,NaN,NaN,NaN,Developing
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SDN,x,NaN,NaN,Developing
5,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Tunisia,788,TUN,NaN,NaN,NaN,Developing
6,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Western Sahara,732,ESH,NaN,NaN,NaN,Developing
7,1,World,2.0,Africa,202.0,Sub-Saharan Africa,14.0,Eastern Africa,British Indian Ocean Territory,86,IOT,NaN,NaN,NaN,Developing
8,1,World,2.0,Africa,202.0,Sub-Saharan Africa,14.0,Eastern Africa,Burundi,108,BDI,x,x,NaN,Developing
9,1,World,2.0,Africa,202.0,Sub-Saharan Africa,14.0,Eastern Africa,Comoros,174,COM,x,NaN,x,Developing


In [142]:
df_pop.shape

(280932, 10)

In [143]:
df_pop.head(10)

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486
5,4,Afghanistan,2,Medium,1955,1955.5,4318.945,3952.047,8270.992,12.669
6,4,Afghanistan,2,Medium,1956,1956.5,4375.800,4023.073,8398.873,12.865
7,4,Afghanistan,2,Medium,1957,1957.5,4437.157,4098.000,8535.157,13.073
8,4,Afghanistan,2,Medium,1958,1958.5,4503.156,4176.941,8680.097,13.295
9,4,Afghanistan,2,Medium,1959,1959.5,4573.914,4260.033,8833.947,13.531


## Data cleaning and wrangling

* Need to harmonize country names so that tests can be matched with cases
* Need to group into regions
* Keep only relevant columns

### Merging tests, cases, regions, and population

* For simplicity, we will only operate on a national level, since testing data is so heterogeneous

0. Drop unnecessary rows and columns
1. Harmonize all country names
2. Aggregate country statistics
3. Merge dataframes

In [145]:
df_tests = (df_tests[['Entity', 'Total COVID-19 tests']]
            .rename(columns={'Entity': 'country', 'Total COVID-19 tests': 'tests'}))
df_cases = (df_cases[['date', 'location', 'new_cases', 'new_deaths', 'total_cases', 'total_deaths']]
            .rename(columns={'location': 'country'}))

In [146]:
df_tests['country'] = df_tests['country'].apply(lambda x: x.split(' - ')[0])

In [155]:
def compare_country_lists(li1, li2):
    return sorted([c for c in set(li1) if c not in set(li2)])

In [156]:
compare_country_lists(df_cases['country'], df_tests['country'])

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Antigua and Barbuda',
 'Argentina',
 'Azerbaijan',
 'Bahamas',
 'Bangladesh',
 'Barbados',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Cambodia',
 'Cameroon',
 'Central African Republic',
 'Chile',
 'Congo',
 'Costa Rica',
 "Cote d'Ivoire",
 'Cuba',
 'Cyprus',
 'Democratic Republic of Congo',
 'Djibouti',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Ethiopia',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guam',
 'Guatemala',
 'Guinea',
 'Guyana',
 'Honduras',
 'Indonesia',
 'International',
 'Iran',
 'Iraq',
 'Jamaica',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kosovo',
 'Lebanon',
 'Liberia',
 'Liechtenstein',
 'Luxembourg',
 'Macedonia',
 'Maldives',
 'Mauritania',
 'Mexico',
 'Moldova',
 'Monaco',
 'Mongolia',
 'Montenegro',
 'Morocco',
 'Myanmar',
 'Namibia',
 'Nepal',
 'Nicaragua',
 'Nigeria

In [157]:
compare_country_lists(df_tests['country'], df_cases['country'])

['Hong Kong']

In [172]:
# Merging
df_cases_tests = pd.merge(df_cases, df_tests, how='left', on='country')

In [174]:
df_regions = (df_regions.rename(columns={'Region Name': 'region', 'Sub-region Name': 'subregion',
                                 'Country or Area': 'country', 'ISO-alpha3 Code': 'ISO3'})
              .filter(['region', 'subregion', 'country', 'ISO3'], axis=1))

In [175]:
df_cases_tests

,date,country,new_cases,new_deaths,total_cases,total_deaths,tests
0,2019-12-31,Afghanistan,0,0,0,0,NaN
1,2020-01-01,Afghanistan,0,0,0,0,NaN
2,2020-01-02,Afghanistan,0,0,0,0,NaN
3,2020-01-03,Afghanistan,0,0,0,0,NaN
4,2020-01-04,Afghanistan,0,0,0,0,NaN
...,...,...,...,...,...,...,...
6633,2020-03-15,World,8140,354,151363,5761,NaN
6634,2020-03-16,World,16051,746,167414,6507,NaN
6635,2020-03-17,World,12745,596,180159,7103,NaN
6636,2020-03-18,World,14750,770,194909,7873,NaN


In [177]:
compare_country_lists(df_cases_tests['country'], df_regions['country'])

['Bolivia',
 'Brunei',
 "Cote d'Ivoire",
 'Czech Republic',
 'Democratic Republic of Congo',
 'International',
 'Iran',
 'Kosovo',
 'Macedonia',
 'Moldova',
 'Palestine',
 'Russia',
 'South Korea',
 'Swaziland',
 'Taiwan',
 'Tanzania',
 'United Kingdom',
 'United States',
 'Vatican',
 'Venezuela',
 'Vietnam',
 'World']

In [178]:
compare_country_lists(df_regions['country'], df_cases_tests['country'])

['American Samoa',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Aruba',
 'Belize',
 'Bermuda',
 'Bolivia (Plurinational State of)',
 'Bonaire, Sint Eustatius and Saba',
 'Botswana',
 'Bouvet Island',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Burundi',
 'Cabo Verde',
 'Cayman Islands',
 'Chad',
 'China, Hong Kong Special Administrative Region',
 'China, Macao Special Administrative Region',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Comoros',
 'Cook Islands',
 'Curaçao',
 'Czechia',
 'Côte d’Ivoire',
 "Democratic People's Republic of Korea",
 'Democratic Republic of the Congo',
 'Dominica',
 'Eritrea',
 'Eswatini',
 'Falkland Islands (Malvinas)',
 'Faroe Islands',
 'Fiji',
 'French Guiana',
 'French Southern Territories',
 'Gibraltar',
 'Greenland',
 'Grenada',
 'Guadeloupe',
 'Guernsey',
 'Guinea-Bissau',
 'Haiti',
 'Heard Island and McDonald Islands',
 'Holy See',
 'Iran (Islamic Republic of)',
 'Isle of Man',
 'Jersey',
 'Kiribati',
 "L

In [188]:
def copy_country_row(df, orig_country, new_country, new_iso):
    row = df[df['country'] == orig_country].copy()
    row['country'] = new_country
    row['ISO3'] = new_iso
    return row

In [197]:
new_rows = []
for old_country, new_country, new_iso in [('China', 'Taiwan', 'TAI'), ('Serbia', 'Kosovo', 'KOS')]:
    new_rows.append(copy_country_row(df_regions, old_country, new_country, new_iso))
new_rows = pd.concat(new_rows)

In [198]:
new_rows

,region,subregion,country,ISO3
123,Asia,Eastern Asia,Taiwan,TAI
208,Europe,Southern Europe,Kosovo,KOS


In [199]:
df_regions = df_regions.append(new_rows)

In [225]:
with Path('data/country_lookup.json').open() as f:
    country_name_dict = json.load(f)

In [205]:
def clean_country_names(df, country_name_dict):
    df = df.copy()
    df['country'] = df['country'].apply(lambda x: country_name_dict.get(x, x))
    return df

df_cases_tests = clean_country_names(df_cases_tests, country_name_dict)
df_regions = clean_country_names(df_regions, country_name_dict)

In [206]:
df_regions

,region,subregion,country,ISO3
0,Africa,Northern Africa,Algeria,DZA
1,Africa,Northern Africa,Egypt,EGY
2,Africa,Northern Africa,Libya,LBY
3,Africa,Northern Africa,Morocco,MAR
4,Africa,Northern Africa,Sudan,SDN
...,...,...,...,...
246,Oceania,Polynesia,Tonga,TON
247,Oceania,Polynesia,Tuvalu,TUV
248,Oceania,Polynesia,Wallis and Futuna Islands,WLF
123,Asia,Eastern Asia,Taiwan,TAI


In [208]:
df_cases_tests_reg = pd.merge(df_cases_tests, df_regions, on='country', how='left')

In [211]:
df_cases_tests_reg = df_cases_tests_reg.query('country not in ["World", "International"]')

In [212]:
df_cases_tests_reg

,date,country,new_cases,new_deaths,total_cases,total_deaths,tests,region,subregion,ISO3
0,2019-12-31,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG
1,2020-01-01,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG
2,2020-01-02,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG
3,2020-01-03,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG
4,2020-01-04,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG
...,...,...,...,...,...,...,...,...,...,...
6553,2020-03-16,Vietnam,4,0,57,0,9696.0,Asia,South-eastern Asia,VNM
6554,2020-03-17,Vietnam,4,0,61,0,9696.0,Asia,South-eastern Asia,VNM
6555,2020-03-18,Vietnam,0,0,61,0,9696.0,Asia,South-eastern Asia,VNM
6556,2020-03-19,Vietnam,15,0,76,0,9696.0,Asia,South-eastern Asia,VNM


### Population

In [217]:
df_pop = (df_pop.rename(columns={'Location': 'country', 'PopTotal': 'population_total', 'Time': 'year'})
                .query('year == 2019')
                .filter(['country', 'population_total', 'year'], axis=1))

In [218]:
df_pop

,country,population_total,year
69,Afghanistan,38041.757,2019
953,Africa,1308064.176,2019
1837,African Group,1306320.572,2019
1988,African Union,1306903.030,2019
2139,African Union: Central Africa,154013.705,2019
...,...,...,...
277314,World,7713468.205,2019
278198,World Bank Regional Groups (developing only),6452517.055,2019
278349,Yemen,29161.922,2019
279233,Zambia,17861.034,2019


In [226]:
df_pop = clean_country_names(df_pop, country_name_dict)

In [227]:
compare_country_lists(df_cases_tests_reg['country'], df_pop['country'])

['Kosovo', 'Taiwan']

In [228]:
new_pop_data = [
    ('Kosovo', 1831, 2019),
    ('Taiwan', 23780, 2019)
]
new_pop_data = pd.DataFrame(new_pop_data, columns=['country', 'population_total', 'year'])

In [229]:
new_pop_data

,country,population_total,year
0,Kosovo,1831,2019
1,Taiwan,23780,2019


In [230]:
df_pop = df_pop.append(new_pop_data)

In [232]:
df_pop['population_total'] = df_pop['population_total'] * 1000

In [233]:
df_pop

,country,population_total,year
69,Afghanistan,3.804176e+07,2019
953,Africa,1.308064e+09,2019
1837,African Group,1.306321e+09,2019
1988,African Union,1.306903e+09,2019
2139,African Union: Central Africa,1.540137e+08,2019
...,...,...,...
278349,Yemen,2.916192e+07,2019
279233,Zambia,1.786103e+07,2019
280117,Zimbabwe,1.464547e+07,2019
0,Kosovo,1.831000e+06,2019


In [234]:
df_cases_tests_reg_pop = pd.merge(df_cases_tests_reg, df_pop, on='country', how='left')

In [235]:
df_cases_tests_reg_pop

,date,country,new_cases,new_deaths,total_cases,total_deaths,tests,region,subregion,ISO3,population_total,year
0,2019-12-31,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG,38041757.0,2019
1,2020-01-01,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG,38041757.0,2019
2,2020-01-02,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG,38041757.0,2019
3,2020-01-03,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG,38041757.0,2019
4,2020-01-04,Afghanistan,0,0,0,0,NaN,Asia,Southern Asia,AFG,38041757.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...
6489,2020-03-16,Vietnam,4,0,57,0,9696.0,Asia,South-eastern Asia,VNM,96462108.0,2019
6490,2020-03-17,Vietnam,4,0,61,0,9696.0,Asia,South-eastern Asia,VNM,96462108.0,2019
6491,2020-03-18,Vietnam,0,0,61,0,9696.0,Asia,South-eastern Asia,VNM,96462108.0,2019
6492,2020-03-19,Vietnam,15,0,76,0,9696.0,Asia,South-eastern Asia,VNM,96462108.0,2019


In [236]:
df_cases_tests_reg_pop = df_cases_tests_reg_pop.assign(
    case_prev=lambda df: df['total_cases'] / df['population_total'],
    case_inc=lambda df: df['new_cases'] / df['population_total'],
    test_rate=lambda df: df['tests'] / df['population_total']
)

In [240]:
df_cases_tests_reg_pop.to_csv('data/full_dataset.csv', index=False)